In [104]:
import os
import csv
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [105]:
# Параметры для обработки изображений
img_height = 512
img_width = 512
batch_size = 64

In [106]:
dataset_dir = './datasetTrain'
classes = os.listdir(dataset_dir)

print(classes)

['Aircraft', 'Airplane', 'Willow']


In [107]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    './datasetTrain',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    './datasetValid/',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)


print(train_generator.class_indices)
class_count = train_generator.class_indices

Found 593 images belonging to 3 classes.


Found 593 images belonging to 3 classes.
{'Aircraft': 0, 'Airplane': 1, 'Willow': 2}


In [108]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
# EfficientNetB3 - модель, которая позволяет кушать больше изображений
# Метрика %Map10
base_model.trainable = False

inputs = tf.keras.layers.Input(shape=(512, 512, 3))

x = base_model(inputs, training=False)
# Преобразование каритнок из датасета в набор векторов, чтобы хранить их в моделе, в дальнейшем их необходимо сравнивать
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(64, activation="softmax", name="last_dense")(x)
outputs = tf.keras.layers.Dense(3, activation='softmax')(x)
# На выходе необходимо получать не именя классов, а именно вектора изображений, которые и будут сравниваться
model = tf.keras.models.Model(inputs, outputs)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['Precision'])

In [109]:
if os.path.exists('newModel.keras'):
    model = tf.keras.models.load_model('newModel.keras')
else:
    model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=10,
        validation_data=validation_generator,
        validation_steps=len(validation_generator)
    )

model.save('newModel.keras')

In [110]:
model_load = tf.keras.models.load_model('newModel.keras')

loss, acc = model.evaluate(validation_generator, verbose=2)
print("Validation accuracy:", acc)

c:\Users\adios\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 - 30s - 3s/step - Precision: 0.0000e+00 - loss: 0.8913
Validation accuracy: 0.0


In [111]:
def load_and_preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(512, 512))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_batch = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_batch)

# Загружаем и подготавливаем изображение
new_image = load_and_preprocess_image('plane.jpg')

In [112]:
# Выполняем предсказание
preds = model.predict(new_image)
predicted_class_index = np.argmax(preds)

# Получаем имя класса на основе индекса
predicted_class_name = classes[predicted_class_index]

print("Predicted class:", predicted_class_name)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted class: Airplane


In [113]:
def find_similar_images(class_index, n=10):
    similar_images = []
    # Заменить class_index на class_name
    for root, dirs, files in os.walk(f'dataset/{class_index}'):
        for file in files[:n]:
            image_path = os.path.join(root, file)
            similar_images.append(image_path)
            print(image_path)

    print(similar_images)        
    return similar_images


In [ ]:
# CSV файл
try:
    # Проверяем существование файла и удаляем его, если он существует
    if os.path.exists('./submission.csv'):
        os.remove('./submission.csv')

    with open('./submission.csv', mode='w', newline='', encoding='utf-8') as submission_file:
        writer = csv.writer(submission_file, delimiter=',', quoting=csv.QUOTE_ALL)
        writer.writerow(['image', 'recs'])

        for filename in os.listdir('./datasetValid/'):
            try:
                class_index = train_generator.class_indices.get(filename.split('.')[0].split('_')[-1])
                recs = find_similar_images(class_index, n=10)
                row = [filename, ','.join(recs)]
                writer.writerow(row)
            except Exception as e:
                print(f'Ошибка при обработке файла {filename}: {e}')

    print("Файл submission.csv успешно создан.")
except Exception as e:
    print(f"Произошла ошибка при создании файла submission.csv: {e}")

[]
[]
[]
Файл submission.csv успешно создан.


In [115]:
similar_images = find_similar_images(predicted_class_name)

[]
